<a href="https://colab.research.google.com/github/mpiatek26/Fewshot_text_classification_with_meta_learning_and_BERT/blob/main/task_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import torch
from torch.utils.data import Dataset, TensorDataset

LABEL_MAP = {'positive': 0, 'negative': 1, 0: 'positive', 1: 'negative'}

class MetaTask(Dataset):
    """
    Dataset class for creating meta-learning tasks.
    """

    def __init__(self, examples, num_task, k_support, k_query, tokenizer, max_seq_length=256):
        """
        Initializes the MetaTask dataset.

        :param examples: List of examples, each example is a dict with keys 'domain', 'text', 'label'.
        :param num_task: Number of training tasks.
        :param k_support: Number of support samples per task.
        :param k_query: Number of query samples per task.
        :param tokenizer: Tokenizer for encoding the text.
        :param max_seq_length: Maximum sequence length for tokenization.
        """
        self.examples = examples
        random.shuffle(self.examples)

        self.num_task = num_task
        self.k_support = k_support
        self.k_query = k_query
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

        self.supports, self.queries = self.create_batch(self.num_task)

    def create_batch(self, num_task):
        """
        Creates batches for support and query sets.

        :param num_task: Number of tasks to create.
        :return: Lists of support and query sets.
        """
        supports, queries = [], []

        for _ in range(num_task):
            domain = random.choice(self.examples)['domain']
            domain_examples = [e for e in self.examples if e['domain'] == domain]

            if len(domain_examples) < (self.k_support + self.k_query):
                continue  # Skip if not enough examples

            selected_examples = random.sample(domain_examples, self.k_support + self.k_query)
            supports.append(selected_examples[:self.k_support])
            queries.append(selected_examples[self.k_support:])

        return supports, queries

    def create_feature_set(self, examples):
        """
        Converts examples into a TensorDataset with encoded features.

        :param examples: List of examples to convert.
        :return: TensorDataset of encoded features.
        """
        all_input_ids = torch.zeros((len(examples), self.max_seq_length), dtype=torch.long)
        all_attention_mask = torch.zeros_like(all_input_ids)
        all_segment_ids = torch.zeros_like(all_input_ids)
        all_label_ids = torch.zeros(len(examples), dtype=torch.long)

        for i, example in enumerate(examples):
            encoded = self.tokenizer.encode_plus(example['text'], add_special_tokens=True,
                                                 max_length=self.max_seq_length, padding='max_length',
                                                 truncation=True, return_attention_mask=True)

            all_input_ids[i] = torch.tensor(encoded['input_ids'], dtype=torch.long)
            all_attention_mask[i] = torch.tensor(encoded['attention_mask'], dtype=torch.long)
            all_segment_ids[i] = torch.tensor(encoded['token_type_ids'], dtype=torch.long)
            all_label_ids[i] = LABEL_MAP[example['label']]

        return TensorDataset(all_input_ids, all_attention_mask, all_segment_ids, all_label_ids)

    def __getitem__(self, index):
        """
        Retrieves a task by index.

        :param index: Index of the task.
        :return: Tuple of support and query sets for the task.
        """
        support_set = self.create_feature_set(self.supports[index])
        query_set = self.create_feature_set(self.queries[index])
        return support_set, query_set

    def __len__(self):
        """
        Returns the number of tasks.
        """
        return self.num_task
